# MONGY: Training `PatchTSMixer` on Financial Candlestick Data
## Direct forecasting example

This notebooke demonstrates the usage of a `PatchTSMixer` model for a multivariate time series forecasting task. This notebook has a dependecy on HuggingFace [transformers](https://github.com/huggingface/transformers) repo. For details related to model architecture, refer to the [TSMixer paper](https://arxiv.org/abs/2306.09364).

In [1]:
# Standard
import os
import random

# Third Party
from transformers import (
    EarlyStoppingCallback,
    PatchTSMixerConfig,
    PatchTSMixerForPrediction,
    Trainer,
    TrainingArguments,
)
import numpy as np
import pandas as pd
import torch

# First Party
from tsfm_public.toolkit.dataset import ForecastDFDataset
from tsfm_public.toolkit.time_series_preprocessor import TimeSeriesPreprocessor
from tsfm_public.toolkit.util import select_by_index, train_test_split

In [2]:
# Set seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [3]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="torch.utils.data.dataloader")

## Load and prepare datasets

In the next cell, please adjust the following parameters to suit your application:
- `dataset_path`: path to local .csv file, or web address to a csv file for the data of interest. Data is loaded with pandas, so anything supported by
`pd.read_csv` is supported: (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).
- `timestamp_column`: column name containing timestamp information, use None if there is no such column
- `id_columns`: List of column names specifying the IDs of different time series. If no ID column exists, use []
- `forecast_columns`: List of columns to be modeled
- `context_length`: The amount of historical data used as input to the model. Windows of the input time series data with length equal to
context_length will be extracted from the input dataframe. In the case of a multi-time series dataset, the context windows will be created
so that they are contained within a single time series (i.e., a single ID).
- `forecast_horizon`: Number of time stamps to forecast in future.
- `train_start_index`, `train_end_index`: the start and end indices in the loaded data which delineate the training data.
- `valid_start_index`, `valid_end_index`: the start and end indices in the loaded data which delineate the validation data.
- `test_start_index`, `test_end_index`: the start and end indices in the loaded data which delineate the test data.
- `patch_length`: The patch length for the `PatchTSMixer` model. Recommended to have a value so that `context_length` is divisible by it.
- `num_workers`: Number of dataloder workers in pytorch dataloader.
- `batch_size`: Batch size. 
The data is first loaded into a Pandas dataframe and split into training, validation, and test parts. Then the pandas dataframes are converted
to the appropriate torch dataset needed for training.

In [4]:
# We want to setup our context, horizon, and patch size based on our task. We want to use
# 4 hours of lookback to start, in order to predict the next 5 minutes of candles. Regarding
# patch length, we know that we will want a larger patch size, so we will start with 64 as
# a base case assumption
context_length = 6 * 60 * 4  # This will give us 4 hours of lookback (6 candles per min * 60 min per hour)
forecast_horizon = 6 * 20 # This will give us 20 minutes of predictions
patch_length = 64

In [5]:
# Load the Dataset from the CSV file
DATASET_PATH = "/home/jack/data/10s-candles-2023.csv"
timestamp_col = 't'

data = pd.read_csv(
    DATASET_PATH,
    parse_dates=[timestamp_col]
)


In [ ]:
import gc

gc.collect()

data.drop(columns=["market_state_MarketState.CLOSED", "market_state_MarketState.OPEN", "market_state_MarketState.EXTENDED"], inplace=True)

# Convert float64 columns to float32
float_columns = data.select_dtypes(include=['float64']).columns
data[float_columns] = data[float_columns].astype('float32')

# Convert object columns to category
object_columns = data.select_dtypes(include=['object']).columns
data[object_columns] = data[object_columns].astype('category')

gc.collect()

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31160015 entries, 0 to 31160014
Data columns (total 7 columns):
 #   Column  Dtype              
---  ------  -----              
 0   t       datetime64[ns, UTC]
 1   targ_o  float32            
 2   targ_h  float32            
 3   targ_l  float32            
 4   targ_c  float32            
 5   targ_v  float32            
 6   ticker  category           
dtypes: category(1), datetime64[ns, UTC](1), float32(5)
memory usage: 861.8 MB


In [ ]:
data.head()

,t,targ_o,targ_h,targ_l,targ_c,targ_v,ticker
0,2023-01-03 09:00:00+00:00,130.279999,130.949997,130.279999,130.949997,4233.0,AAPL
1,2023-01-03 09:00:10+00:00,130.979996,131.000000,130.929993,130.929993,744.0,AAPL
2,2023-01-03 09:00:20+00:00,130.979996,131.000000,130.929993,130.929993,0.0,AAPL
3,2023-01-03 09:00:30+00:00,130.979996,131.000000,130.929993,130.929993,0.0,AAPL
4,2023-01-03 09:00:40+00:00,130.979996,130.979996,130.979996,130.979996,223.0,AAPL


## Plan for Cleaning Data 

1. Remove any rows where the market is closed. We will never be using this in the actual model.
2. This should give us a a couple hundred time-series per ticker. We want to create a time-series ID for each continuous time series in the dataset. This will be used as a column ID.
3. Then iterate over each ticker. For each ticker:
 - Use our date indicices, to create a train, validation, and test set for that ticker. Be sure that the date the sets do not slice any time-series (e.g. each of the 3 sets should have a unique set of time-series IDs)
 - Append eacah of the three sets to a master train, validation, and test set respectively. 
 - These steps will replace the current `select_by_index` usage
4. Train the `TimeSeriesPreprocessor` on the train set
5. Create the train, validation, and test datasets, using the trained preprocessor

In [ ]:
# Check for NaN values
data.ffill(inplace=True)
data.isna().sum()

t         0
targ_o    0
targ_h    0
targ_l    0
targ_c    0
targ_v    0
ticker    0
dtype: int64

In [ ]:
train_data, valid_data, test_data = train_test_split(
    data,
    id_columns=['ticker'],
    train=0.8,
    test=0.1,
    valid_test_offset=context_length
)

gc.collect()

train_data.shape, test_data.shape, test_data.shape

/home/jack/.venv/lib/python3.11/site-packages/tsfm_public/toolkit/util.py:221: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groups = df.groupby(_get_groupby_columns(id_columns))


((24928009, 7), (3130398, 7), (3130398, 7))

In [ ]:
# # Before we set the explicit train, validation, and test indicies, let's retrieve these
# # indicies by searching over the 't' column

# # Find the index of the row with the earliest timestamp on October 1, 2023
# oct_15_2023 = '2023-10-15'
# oct_15_2023_index = data[timestamp_col].searchsorted(pd.to_datetime(oct_15_2023).tz_localize('UTC'))

# nov_22_2023 = '2023-11-22'
# nov_22_2023_index = data[timestamp_col].searchsorted(pd.to_datetime(nov_22_2023).tz_localize('UTC'))


In [ ]:
id_columns = ['ticker']
forecast_columns = ["targ_o", "targ_h", "targ_l", "targ_c", "targ_v"]
# train_start_index = None  # None indicates beginning of dataset
# train_end_index = oct_15_2023_index

# # we shift the start of the validation period back by context length so that
# # the first validation timestamp is immediately following the training data
# valid_start_index = train_end_index - context_length
# # we will end the validation set at the same date that the test set begins
# valid_end_index = nov_22_2023_index

# # we shift the start of the test period back by context length so that
# # the first test timestamp is immediately following the validation data
# test_start_index = valid_end_index - context_length
# test_end_index = None # none indicates the end of the dataset

In [ ]:

# train_data = select_by_index(
#     data,
#     id_columns=id_columns,
#     start_index=train_start_index,
#     end_index=train_end_index,
# )
# valid_data = select_by_index(
#     data,
#     id_columns=id_columns,
#     start_index=valid_start_index,
#     end_index=valid_end_index,
# )
# test_data = select_by_index(
#     data,
#     id_columns=id_columns,
#     start_index=test_start_index,
#     end_index=test_end_index,
# )

tsp = TimeSeriesPreprocessor(
    timestamp_column=timestamp_col,
    id_columns=id_columns,
    target_columns=forecast_columns,
    scaling=True,
)
tsp.train(train_data)

gc.collect()

/home/jack/.venv/lib/python3.11/site-packages/tsfm_public/toolkit/time_series_preprocessor.py:508: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grps = df.groupby(self.id_columns)
/home/jack/.venv/lib/python3.11/site-packages/tsfm_public/toolkit/time_series_preprocessor.py:384: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grps = dataset.groupby(by=group_by_columns)


0

In [ ]:
train_dataset = ForecastDFDataset(
    tsp.preprocess(train_data),
    id_columns=id_columns,
    target_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)
valid_dataset = ForecastDFDataset(
    tsp.preprocess(valid_data),
    id_columns=id_columns,
    target_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)
test_dataset = ForecastDFDataset(
    tsp.preprocess(test_data),
    id_columns=id_columns,
    target_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)

# Clean up the underlying dataframes, and garbage collect to keep memory clean
del train_data
del valid_data
del test_data

gc.collect()

/home/jack/.venv/lib/python3.11/site-packages/tsfm_public/toolkit/time_series_preprocessor.py:652: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_out = df.groupby(id_columns, group_keys=False).apply(
/home/jack/.venv/lib/python3.11/site-packages/tsfm_public/toolkit/dataset.py:173: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  self.group_df = data_df.groupby(by=self.id_columns[0])
/home/jack/.venv/lib/python3.11/site-packages/tsfm_public/toolkit/time_series_preprocessor.py:652: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=

0

## Training `PatchTSMixer` From Scratch

Adjust the following model parameters according to need.
- `d_model` (`int`, *optional*, defaults to 8):
    Hidden dimension of the model. Recommended to set it as a multiple of patch_length (i.e. 2-8X of
    patch_len). Larger value indicates more complex model.
- `expansion_factor` (`int`, *optional*, defaults to 2):
    Expansion factor to use inside MLP. Recommended range is 2-5. Larger value indicates more complex model.
- `num_layers` (`int`, *optional*, defaults to 3):
    Number of layers to use. Recommended range is 3-15. Larger value indicates more complex model.
- `mode`: (`str`, either to 'common_channel' or `mix_channel`)

In [ ]:
config = PatchTSMixerConfig(
    context_length=context_length,
    prediction_length=forecast_horizon,
    patch_length=patch_length,
    num_input_channels=len(forecast_columns),
    patch_stride=patch_length,
    d_model=2 * patch_length,
    num_layers=3,
    expansion_factor=4,
    dropout=0.5,
    head_dropout=0.7,
    mode="mix_channel",
    scaling="std",
)
model = PatchTSMixerForPrediction(config=config)

# Training Run Summaries

**Run 1**: (N/A)
This run used the full year of data, and was used as a baseline to establish that the `mix_channel` mode is more effective for our task. Additionally, all subsequent runs have been updated, to instead use only the first three months of data as training data. Thus, while the loss for this run is lower, it is not indicaitve of the paramters being a better fit, just a result of having a larger dataset.

**Run 2** (0.108476):
This run was the first in which only the first two months of data was used as a training set. March was then split in half to form the validation and test sets. Additionally, the context window was expanded, to include the last four hours of data. While this wasn't explicitly compared against a shorter context window with the same dataset, the results of the paper provide an incredibly strong suggestions towards this approach yielding more effective performance.

**Run 3** (0.108230):
This run included involved increasing the `num_layers` argument from 3 to 5. This adds additional layers to the model, giving it more of an ability to percieve complex patterns in the financial data. This results in a larger model, but hopefully, will allow the model to better understand the nuances of the highly complex financial data it is being trained on.

**Run 4**: (0.107247)
This run included further incrementing the `num_layers` argument from 5 to 10. This adds additional further layers to capture more of the complex patterns in the financial dataset. 

_NOTE_: The `num_layers` does not seem to provide additional aid in this trainin task, with the side-effect of signifitcanlty increasing the inference time. As a result, we are making the decision to keep `num_layers = 3`.

---

**Run 5**: (0.108397)
The `num_layers` argument has been reset to a value of 3, which returns our baseline back to _Run 2_. The `expansion_factor` has been increased from 3 to 4. This yeilded a slight decrease in validation loss, so potentially worth running a second experiment, but likely best to test patching instead.

**Run 6** ()

In [ ]:
# Let's clean all the stuff we no longer need
del data

gc.collect()

0

In [17]:
# Compute the run number
run_num = 13
save_dir = f"./checkpoints/run_{run_num}"

# Check if save_dir exists
assert not os.path.exists(save_dir), "Please update the run_num to avoid overwriting checkpoints!"

num_workers = 4  # g4dn instance has 4 vCPUs
batch_size = 256 # Size of each batches sent to GPU
num_steps = 2500

train_args = TrainingArguments(
    output_dir=f"{save_dir}/output/",
    overwrite_output_dir=True,
    learning_rate=0.0001,
    num_train_epochs=100,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=num_steps,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=8,
    eval_accumulation_steps=250,
    dataloader_num_workers=num_workers,
    report_to="tensorboard",
    save_strategy="steps",
    save_steps=num_steps,
    logging_strategy="steps",
    logging_steps=num_steps,
    save_total_limit=3,
    logging_dir=f"{save_dir}/logs/",  # Make sure to specify a logging directory
    load_best_model_at_end=True,  # Load the best model when training ends
    metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
    greater_is_better=False,  # For loss
    label_names=["future_values"], # The names of the "ground truth" values to compare predictions against
)

# Create a new early stopping callback with faster convergence properties
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=5,  # Number of epochs with no improvement after which to stop
    early_stopping_threshold=0.001,  # Minimum improvement required to consider as improvement
)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    callbacks=[early_stopping_callback],
)

print("Doing forecasting training on FULL Dataset")
trainer.train()

Doing forecasting training on FULL Dataset


Step,Training Loss,Validation Loss


RuntimeError: DataLoader worker (pid 21355) is killed by signal: Killed. 

In [106]:
trainer.evaluate(test_dataset)

/home/jack/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


{'eval_loss': 0.2793983221054077,
 'eval_runtime': 32.6588,
 'eval_samples_per_second': 11915.05,
 'eval_steps_per_second': 93.114,
 'epoch': 4.0}

## If we want to train from scratch for a few specific forecast channels